In [1]:
file_path = r'C:\Users\KG\Documents\GitHub\advent-of-code-2023\5\maps.txt' 

with open(file_path, 'r') as file:
    orgiginal_lines = file.readlines()

orgiginal_lines = [line.strip() for line in orgiginal_lines]
orgiginal_lines

['seeds: 564468486 226119074 3264322485 135071952 3144185610 89794463 1560365167 555667043 2419038624 7808461 1264209624 9380035 105823719 425973940 4115473551 104486997 3784956593 300187503 975280918 257129208',
 '',
 'seed-to-soil map:',
 '1383244180 2567207479 366571891',
 '3099137706 184794382 526583321',
 '3625721027 2933779370 589006054',
 '1749816071 838554248 828858952',
 '4214727081 3750906360 80240215',
 '2585129273 2301737819 265469660',
 '919423459 3831146575 463820721',
 '2850598933 2053199046 248538773',
 '184794382 1667413200 385785846',
 '570580228 3522785424 221666686',
 '2578675023 3744452110 6454250',
 '792246914 711377703 127176545',
 '',
 'soil-to-fertilizer map:',
 '1263133301 1101112301 247905608',
 '0 1881569198 406369887',
 '3891730222 4233430396 61536900',
 '2723511851 1349017909 413665324',
 '2115333016 1762683233 118885965',
 '4272202829 4116902795 22764467',
 '1668270240 169908659 417260703',
 '4047030256 3891730222 225172573',
 '1511038909 990025537 111086

In [2]:
seeds = orgiginal_lines[0][len("seeds: "):].split(" ")
seeds

['564468486',
 '226119074',
 '3264322485',
 '135071952',
 '3144185610',
 '89794463',
 '1560365167',
 '555667043',
 '2419038624',
 '7808461',
 '1264209624',
 '9380035',
 '105823719',
 '425973940',
 '4115473551',
 '104486997',
 '3784956593',
 '300187503',
 '975280918',
 '257129208']

In [3]:
maps = {}
new_map = True
ranges = []
for line in orgiginal_lines[2:]:
    if line == "":
        maps[key] = ranges
        new_map = True
        ranges = []
    else:
        if new_map:
            w = line[:-len(" map:")].split("-")
            key = (w[0], w[-1])
            new_map = False
        else:
            ranges.append([int(n) for n in line.split(" ")])
else:
    maps[key] = ranges
maps


{('seed', 'soil'): [[1383244180, 2567207479, 366571891],
  [3099137706, 184794382, 526583321],
  [3625721027, 2933779370, 589006054],
  [1749816071, 838554248, 828858952],
  [4214727081, 3750906360, 80240215],
  [2585129273, 2301737819, 265469660],
  [919423459, 3831146575, 463820721],
  [2850598933, 2053199046, 248538773],
  [184794382, 1667413200, 385785846],
  [570580228, 3522785424, 221666686],
  [2578675023, 3744452110, 6454250],
  [792246914, 711377703, 127176545]],
 ('soil', 'fertilizer'): [[1263133301, 1101112301, 247905608],
  [0, 1881569198, 406369887],
  [3891730222, 4233430396, 61536900],
  [2723511851, 1349017909, 413665324],
  [2115333016, 1762683233, 118885965],
  [4272202829, 4116902795, 22764467],
  [1668270240, 169908659, 417260703],
  [4047030256, 3891730222, 225172573],
  [1511038909, 990025537, 111086764],
  [3953267122, 4139667262, 93763134],
  [2085530943, 960223464, 29802073],
  [1622125673, 2601739738, 46144567],
  [2234218981, 2647884305, 489292870],
  [481920

In [4]:
def map_shit(numb, maps):
    for map in maps:
        if map[1] <= numb and numb <=  map[1] + map[2] - 1:
            return numb - map[1] + map[0]
    return numb
map_shit(98, maps[("seed","soil")])

98

In [5]:
numbs = [int(s) for s in seeds]
for name, map in maps.items():
    new_numbs = []
    for numb in numbs:
        new_numbs.append(map_shit(numb, map))
    numbs = new_numbs
numbs


[2422809993,
 3367027469,
 697158568,
 1136789458,
 2725834042,
 3527673368,
 1195534841,
 2414008550,
 517243940,
 3859069353,
 601644709,
 3860640927,
 2408170138,
 1585436691,
 1291781871,
 2406833416,
 340994526,
 1648034705,
 870127997,
 1604976410]

In [6]:
min(numbs)

340994526

In [7]:
start, length = tuple([int(s) for s in seeds][:2])
r = range(start, start + length)
len(r) == length


True

In [8]:
1+1

2

In [9]:
import torch

In [10]:
def map_range_shit(numbs, maps):
    done = torch.tensor([], dtype=torch.int64).to("cuda")
    not_done = numbs.clone().to("cuda")

    for map in maps:
        in_ranged = not_done[(map[1] <= not_done) & (not_done <=  (map[1] + map[2] - 1))]
        not_done = not_done[~((map[1] <= not_done) &( not_done <=  (map[1] + map[2] - 1)))]
        done = torch.cat((done, ((in_ranged - map[1]) + map[0]))) 
    return torch.cat((done, not_done))
   


In [11]:
rs = [int(s) for s in seeds]
"""int(map_range_shit(torch.tensor([rs[0]]), maps[("seed","soil")]))
map_shit(rs[0], maps[("seed","soil")])"""

assert sorted([int(map_range_shit(torch.tensor([r], dtype=torch.int64),  maps[("seed","soil")])) for r in rs]) == sorted([int(n) for n in map_range_shit(torch.tensor(rs),  maps[("seed","soil")])])

assert sorted([int(n) for n in map_range_shit(torch.tensor(rs),  maps[("seed","soil")])]) == sorted([map_shit(r, maps[("seed","soil")]) for r in rs])


In [12]:
min(range(10))

0

3140462398

In [13]:
torch.cuda.is_available()

True

In [14]:
"""rs = [int(s) for s in seeds]
smalest = ""
for i in range(0,len(rs),2):
    print(i)
    ar = torch.tensor(range(rs[i], rs[i] + rs[i+1]), dtype=torch.int64).to("cuda")
    for name, map in maps.items():
        print(name)
        new_ar = torch.tensor([], dtype=torch.int64).to("cuda")
        for seed in ar:
            new_ar = torch.cat((new_ar, torch.tensor([map_shit(seed, map)],  dtype=torch.int64).to("cuda"))) 
        #ar = map_range_shit(ar, map)
        ar = new_ar
    if smalest == "":
        smalest = torch.min(ar)
    else:
        ar_min = int(torch.min(ar))
        smalest = min(smalest, ar_min)
smalest"""

'rs = [int(s) for s in seeds]\nsmalest = ""\nfor i in range(0,len(rs),2):\n    print(i)\n    ar = torch.tensor(range(rs[i], rs[i] + rs[i+1]), dtype=torch.int64).to("cuda")\n    for name, map in maps.items():\n        print(name)\n        new_ar = torch.tensor([], dtype=torch.int64).to("cuda")\n        for seed in ar:\n            new_ar = torch.cat((new_ar, torch.tensor([map_shit(seed, map)],  dtype=torch.int64).to("cuda"))) \n        #ar = map_range_shit(ar, map)\n        ar = new_ar\n    if smalest == "":\n        smalest = torch.min(ar)\n    else:\n        ar_min = int(torch.min(ar))\n        smalest = min(smalest, ar_min)\nsmalest'

In [15]:
def split_list(input_list, num_parts):
    avg = len(input_list) // num_parts
    remainder = len(input_list) % num_parts
    return [input_list[i * avg + min(i, remainder):(i + 1) * avg + min(i + 1, remainder)] for i in range(num_parts)]


In [16]:
28264884 * 4

113059536

In [17]:
import math
inte_seeds = [int(s) for s in seeds]
smalest = ""
for i in range(0,len(inte_seeds),2):
    print(i)
    seed_range = range(inte_seeds[i], inte_seeds[i] + inte_seeds[i+1])


    partition = math.ceil((len(seed_range) / 23059536)  )
    print("part", partition)
    ranges = split_list(seed_range, partition)
    for r in ranges:
        ar = torch.arange(min(r), max(r), dtype=torch.int64).to("cuda")
        for name, map in maps.items():
            ar = map_range_shit(ar, map)
        if smalest == "":
            smalest = torch.min(ar)
        else:
            ar_min = int(torch.min(ar))
            smalest = min(smalest, ar_min)
smalest


0
part 10
2
part 6
4
part 4
6
part 25
8
part 1
10
part 1
12
part 19
14
part 5
16
part 14
18
part 12


52210644